In [49]:
import pandas as pd
import numpy as np
from stravalib import Client
import webbrowser
from Pace import Pace
from units import *
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import webbrowser
from functions import *
from tqdm import tqdm
import pickle

In [50]:
client = Client()
client_id = 145799
client_secret = 'e602b60cc9fffa0a39c9a1c45589c87ea1f11e1b'
redirect_uri = 'http://localhost:8282/authorized'

## Generate authorization code

In [51]:
def getCode():
    authorize_url = client.authorization_url(client_id=client_id, 
                                             redirect_uri=redirect_uri,
                                             scope=['read_all', 'profile:read_all', 'activity:read_all'])
    webbrowser.open(authorize_url)
    code = input("Enter the code: ")
    return code

In [48]:
# read the client from the pickle file
try:
    client = pickle.load(open(PKL, 'rb'))
    print('Client loaded from file')
except:
    print('You must authenticate - run below cell.')

Client loaded from file


In [52]:
import pickle

if client.access_token is None:    
    authorize_url = client.authorization_url(client_id=client_id, 
                                             redirect_uri=redirect_uri,
                                             scope=['read_all', 'profile:read_all', 'activity:read_all'])
    
    # Open the authorization URL in a web browser
    webbrowser.open(authorize_url)
    
    # After authorization, you'll be redirected to a URL. Copy the 'code' parameter from this URL
    code = input("Enter the code: ")
    
    # Exchange the code for a token
    token_response = client.exchange_code_for_token(client_id=client_id, 
                                                    client_secret=client_secret, 
                                                    code=code)
    
    # Now you can use the access token to make API requests
    access_token = token_response['access_token']
    client.access_token = access_token

print('Authenticated')

No rates present in response headers


Authenticated


In [53]:
with open(PKL, 'wb') as pkl:
    pickle.dump(client, pkl)
print('Client saved to file. Access code: ', client.access_token)

Client saved to file. Access code:  67ca12bf2eecb4eb3df05a4c9bba38a796860c14


## Get Athlete Profile

In [41]:
myActivities = pd.read_csv('myActivities.csv')

In [64]:
activities = client.get_activities()
activity_list = []
for activity in activities:
    activity_dict = {
        'id': activity.id,
        'name': activity.name,
        'start_date': activity.start_date,
        'distance': activity.distance,  # Convert distance to numeric (meters)
        'moving_time': activity.moving_time,  # Convert to seconds
        'elapsed_time': activity.elapsed_time,  # Convert to seconds
        'total_elevation_gain': activity.total_elevation_gain,
        'type': activity.type,
        'average_speed': activity.average_speed if activity.average_speed else None,  # Convert to numeric
        'max_speed': activity.max_speed if activity.max_speed else None,  # Convert to numeric
        'average_heartrate': activity.average_heartrate,
        'max_heartrate': activity.max_heartrate,
    }
    activity_list.append(activity_dict)
    
myActivities = pd.DataFrame(activity_list)
everything = myActivities.copy()
myActivities = myActivities[myActivities['type'].astype(str).apply(lambda x: 'Run' in x)]
myActivities.head()

,id,name,start_date,distance,moving_time,elapsed_time,total_elevation_gain,type,average_speed,max_speed,average_heartrate,max_heartrate
0,13431683380,2-1 Tempo Morning 10k,2025-01-23 13:00:19+00:00,10043.6,3905,3930,0.0,root='Run',2.572,3.45,170.1,187.0
2,13415033575,K200s swap from treadmill to outside,2025-01-21 15:58:33+00:00,2641.2,1192,1458,0.0,root='Run',2.216,3.58,156.9,177.0
3,13415559185,K200s outside,2025-01-21 16:25:03+00:00,9064.2,3405,3506,37.9,root='Run',2.662,5.16,171.8,187.0
4,13399628273,Everything is bigger in Texas,2025-01-19 16:38:43+00:00,13023.6,5079,5079,43.4,root='Run',2.564,4.96,171.9,180.0
5,13382848028,4.5mi Easy Run with Runna ✅,2025-01-17 23:02:43+00:00,5078.0,2178,2206,7.3,root='Run',2.331,5.58,151.7,164.0


In [65]:
print(f'Loaded {len(myActivities)} activities from API')

Loaded 41 activities from API


## Get Activity Data

In [66]:
def get_activity_streams(client, activity_id, resolution='high'):
    """
    Fetch heart rate, pace (velocity), and elevation streams for a given activity ID.
    """
    streams = client.get_activity_streams(
        activity_id,
        types=['heartrate', 'velocity_smooth', 'altitude', 'time', 'distance'],
        resolution=resolution,
    )
    
    heart_rate = streams['heartrate'].data if 'heartrate' in streams else None
    velocity = streams['velocity_smooth'].data if 'velocity_smooth' in streams else None
    elevation = streams['altitude'].data if 'altitude' in streams else None
    time_index = streams['time'].data if 'time' in streams else None
    distance_index = streams['distance'].data if 'distance' in streams else None
    return heart_rate, velocity, elevation, time_index, distance_index

In [67]:
myActivities['granular data'] = myActivities['id'].apply(lambda x: get_activity_streams(client, x, resolution='high'))

In [136]:
# Change this to whatever row number you want
rowId = 1

things = myActivities.iloc[rowId]
activityId = things['id']
name = things['name']
print(name)
hr, pace, elevation, timeIdx, distanceIdx = get_activity_streams(client, activityId, resolution='high')

# Convert to imperial system
elevation = [int(round(metersToFeet(elev), 0)) for elev in elevation]
timeIdx = [i/60 for i in timeIdx]
distanceIdx = [round(metersToMiles(i), 2) for i in distanceIdx]

pace = [Pace.from_mps(v) for v in pace if v > 0]

K200s swap from treadmill to outside


## Interactive Plots

In [142]:
def numericPlot(base, items):
    data = pd.DataFrame({
        base: items,
        'Time': timeIdx,
        'Distance': distanceIdx
    })
    return data

### Pace

In [143]:
diff = len(timeIdx) - len(pace)
average = sum(pace)/len(pace)
x = pace + [average] * diff

stuff = numericPlot('Pace', x)
myPaces = stuff['Pace']
index = stuff['Time']

y_values = [pace.time/60 for pace in myPaces]
labels = [str(pace) for pace in myPaces]

In [144]:
import plotly.graph_objects as go

# Create the plot
fig = go.Figure(data=go.Scatter(
    x=index,
    y=y_values,
    mode='markers+lines',
    text=[f"{str(pace)}, {hr[i]} BPM" for i, pace in enumerate(myPaces)],
    marker=dict(color=hr, colorscale='solar', colorbar=dict(title='Heart Rate'))
))

fig.update_layout(
    title=f'Pace Plot: {name}',
    xaxis_title='Time',
    yaxis_title='Pace (min/mi)'
)
fig.show()

In [145]:
index = stuff['Distance']
y_values = [pace.time/60 for pace in myPaces]
labels = [str(pace) for pace in myPaces]

# Create the plot
fig = go.Figure(data=go.Scatter(
    x=index,
    y=y_values,
    mode='markers+lines',
    text=[f"{str(pace)}, {hr[i]} BPM" for i, pace in enumerate(myPaces)],
    marker=dict(color=hr, colorscale='solar', colorbar=dict(title='Heart Rate'))
))

fig.update_layout(
    title='Pace Plot',
    xaxis_title='Distance',
    yaxis_title='Pace (min/mi)'
)
fig.show()

### HR

In [146]:
zones = client.get_athlete_zones().dict()
values = zones['heart_rate']['zones'][:-1]

In [147]:
def getZone(hr):
    n = len(values)
    assert hr >= 0, 'HR must be non-negative'
    for i in range(n):
        bucket = values[i]
        if bucket['min'] <= hr <= bucket['max']:
            return f'Zone {i+1}'
    raise ValueError(f'HR {hr} is out of range')

In [148]:
base = 'HR'
data = numericPlot(base, hr)
try:
    data['Zone']
except:
    data['Zone'] = data[base].apply(getZone)
fig = go.Figure()

In [149]:
zone_colors = {
    'Zone 1': 'green',
    'Zone 2': 'yellow',
    'Zone 3': 'orange',
    'Zone 4': 'red'
}

# Create the figure
fig = go.Figure()

# Add traces for each zone
for zone in data['Zone'].unique():
    zone_data = data[data['Zone'] == zone]
    fig.add_trace(go.Scatter(
        x=zone_data['Time'],
        y=zone_data['HR'],
        mode='lines',
        name=zone,
        line=dict(color=zone_colors[zone], width=1)  # Assign color based on zone
    ))

fig.update_layout(
    title=f'{base}: {name}',
    xaxis_title='Time',
    yaxis_title=base,
    autosize=False,
    width=800,
    height=600
)

fig.show()
print(data.columns)

Index(['HR', 'Time', 'Distance', 'Zone'], dtype='object')


In [150]:
# Create the figure
fig = go.Figure()

# Add traces for each zone using Distance as the x-axis
for zone in data['Zone'].unique():
    zone_data = data[data['Zone'] == zone]
    fig.add_trace(go.Scatter(
        x=zone_data['Distance'],
        y=zone_data['HR'],
        mode='lines',
        name=zone,
        line=dict(color=zone_colors[zone], width=1)  # Assign color based on zone
    ))

fig.update_layout(
    title=f'{base}: {name}',
    xaxis_title='Distance',
    yaxis_title=base,
    autosize=False,
    width=800,
    height=600
)
print(data.columns)
fig.show()

Index(['HR', 'Time', 'Distance', 'Zone'], dtype='object')


In [129]:
data['Pace'] = x

grouped = data.groupby('Zone').mean()
minimums = data.groupby('Zone').min()
maximums = data.groupby('Zone').max()
minimums['Pace'] = minimums['Pace'].apply(lambda x: x.time).apply(lambda j: Pace.fromSeconds(j))
maximums['Pace'] = maximums['Pace'].apply(lambda x: x.time).apply(lambda j: Pace.fromSeconds(j))
for z in grouped.index:
    print(f'{z} Average Pace: {Pace.fromSeconds(grouped.loc[z, "Pace"])}')

Zone 1 Average Pace: 153:08/mi
Zone 2 Average Pace: 38:40/mi
Zone 3 Average Pace: 19:16/mi


In [151]:
print(data.columns)

Index(['HR', 'Time', 'Distance', 'Zone'], dtype='object')


In [152]:
for z in minimums.index:
    print(f'{z} Range: {minimums.loc[z, "Pace"]} to {maximums.loc[z, "Pace"]}')

Zone 1 Range: 1341:07/mi to 8:01/mi
Zone 2 Range: 191:35/mi to 7:29/mi
Zone 3 Range: 1277:15/mi to 8:22/mi


In [155]:
data['Pace'] = x
data['HR'] = hr
data['PaceTime'] = data['Pace'].apply(lambda x: x.time)
data['PaceStr'] = data['Pace'].astype(str)

In [156]:
import pandas as pd
import plotly.graph_objects as go

# Assuming `data` is your DataFrame with the following columns:
# 'Zone' (Heart Rate Zone), 'Distance', 'PaceTime' (numeric pace in seconds), 'PaceStr' (string representation of pace)

# Function to filter out outliers using the 1.5 x IQR rule
def exclude_outliers(df, column):
    q1 = df[column].quantile(0.25)  # First quartile (25th percentile)
    q3 = df[column].quantile(0.75)  # Third quartile (75th percentile)
    iqr = q3-q1
    lower_fence = q1 - 1.5 * iqr
    upper_fence = q3 + 1.5 * iqr
    return df[(df[column] >= lower_fence) & (df[column] <= upper_fence)]  # Filter data within fences

# Apply the outlier exclusion function to each zone
filtered_data = pd.concat([
    exclude_outliers(data[data['Zone'] == zone], 'PaceTime')
    for zone in data['Zone'].unique()
])

# Create the boxplot
fig = go.Figure()

# Add a box trace for each heart rate zone
for zone in filtered_data['Zone'].unique():
    zone_data = filtered_data[filtered_data['Zone'] == zone]
    fig.add_trace(go.Box(
        y=zone_data['PaceTime'],  # Numeric pace values for the boxplot
        x=[zone] * len(zone_data),  # Use the zone as the x-axis category
        name=zone,  # Name of the trace (zone)
        text=zone_data['PaceStr'],  # Tooltip labels (string representation of pace)
        hovertemplate='Zone: %{x}<br>Pace: %{text}<extra></extra>',  # Custom tooltip format
        boxmean=True  # Show mean as a dashed line
    ))

# Update layout for better visualization
fig.update_layout(
    title='Pace Distribution by Heart Rate Zone (Outliers Excluded)',
    xaxis_title='Heart Rate Zone',
    yaxis_title='Pace (seconds)',
    autosize=False,
    width=800,
    height=600
)

# Show the plot
fig.show()

### Elevation

In [157]:
data = numericPlot('Elevation', elevation)
fig = px.line(data, x='Time', y='Elevation', title=f'Elevation: {name}')
fig.update_layout(autosize=False, width=800, height=600)
fig.show()

In [158]:
fig = px.line(data, x='Distance', y='Elevation', title=f'Elevation: {name}')
fig.update_layout(autosize=False, width=800, height=600)
fig.show()

In [159]:
metersToFeet(max(elevation)) - metersToFeet(min(elevation))

0.0

In [160]:
metersToFeet(float(things['total_elevation_gain']))

0.0

## HR-Pace Correlation

In [161]:
data['Pace'] = x
data['HR'] = hr
data['PaceTime'] = data['Pace'].apply(lambda x: x.time/60)
data['PaceStr'] = data['Pace'].astype(str)

In [164]:
x = data[['HR', 'PaceTime']]
x['PaceTime'] = x['PaceTime'].apply(lambda a: a/60)
correlation = x.corr()['HR']['PaceTime']
float(correlation)

-0.23637622518871373

In [163]:
fig = px.scatter(x, x='HR', y='PaceTime', title='Scatter Plot of HR by Pace')
fig.show()

## Elevation Gradient

In [117]:
def gradient(arr):
    slopes = []
    for i in range(len(arr)):
        if i == 0:
            rise = i
        else:
            rise = arr[i] - arr[i-1]
        slopes.append(rise)
    return slopes
gradient([1,2,3,-3,1])

[0, 1, 1, -6, 4]

In [118]:
gradient(elevation)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
